## clean up LOTUS and save it in a new file

In [2]:
import polars as pl
import numpy as np
from defl import * 


In [3]:
df = load_lotus_csv("../data/LOTUS/230106_frozen_metadata.csv.gz")

df

ColumnNotFoundError: organism_taxonomy_gbifid

In [ ]:
df.write_csv("../example_data/230106_frozen_metadata_cleaned.csv")

## save all compounds in a list as "reactant" and "product"

In [1]:
%%time

from pymongo import MongoClient
import polars as pl

# Set the maximum number of characters displayed per column
pl.Config.set_tbl_rows(100)
pl.Config(fmt_str_lengths=700)

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/') 
db = client['lotus_mines_enzymatic'] 
reactions_collection = db['reactions'] 
    
pipeline = [
    {
        '$project': {
            '_id': 1, 
            'reactants': {
                '$map': {
                    'input': '$Reactants', 
                    'as': 'reactant', 
                    'in': {
                        '$arrayElemAt': [
                            '$$reactant', 1
                        ]
                    }
                }
            }, 
            'products': {
                '$map': {
                    'input': '$Products', 
                    'as': 'product', 
                    'in': {
                        '$arrayElemAt': [
                            '$$product', 1
                        ]
                    }
                }
            }
        }
    }, {
        '$lookup': {
            'from': 'compounds', 
            'localField': 'reactants', 
            'foreignField': '_id', 
            'as': 'reactant_details'
        }
    }, {
        '$lookup': {
            'from': 'compounds', 
            'localField': 'products', 
            'foreignField': '_id', 
            'as': 'product_details'
        }
    }, {
        '$addFields': {
            'reactants': {
                '$map': {
                    'input': '$reactant_details', 
                    'as': 'detail', 
                    'in': {
                        'id': '$$detail._id', 
                        'type': '$$detail.Type'
                    }
                }
            }, 
            'products': {
                '$map': {
                    'input': '$product_details', 
                    'as': 'detail', 
                    'in': {
                        'id': '$$detail._id', 
                        'type': '$$detail.Type'
                    }
                }
            }
        }
    }, {
        '$project': {
            'reactant_details': 0, 
            'product_details': 0
        }
    }, {
        '$addFields': {
            'starting_compounds': {
                '$filter': {
                    'input': '$reactants', 
                    'as': 'reactant', 
                    'cond': {
                        '$eq': [
                            '$$reactant.type', 'Starting Compound'
                        ]
                    }
                }
            }, 
            'predicted_compounds': {
                '$filter': {
                    'input': '$products', 
                    'as': 'product', 
                    'cond': {
                        '$eq': [
                            '$$product.type', 'Predicted'
                        ]
                    }
                }
            }
        }
    }, {
        '$project': {
            '_id': 1, 
            'starting_compounds': '$starting_compounds.id', 
            'predicted_compounds': '$predicted_compounds.id'
        }
    }
]

CPU times: user 77.8 ms, sys: 15.3 ms, total: 93.1 ms
Wall time: 91.2 ms


In [2]:
%%time

# Execute the aggregation pipeline
cursor = reactions_collection.aggregate(pipeline)
data = list(cursor)

# Convert to a Polars DataFrame
df = pl.DataFrame(data)

# rename the "_id" to be more specific
df = df.rename({"_id": "reaction_id"})

# Show the DataFrame
print(df)

shape: (3_435_078, 3)
┌────────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│ reaction_id                    ┆ starting_compounds             ┆ predicted_compounds            │
│ ---                            ┆ ---                            ┆ ---                            │
│ str                            ┆ list[str]                      ┆ list[str]                      │
╞════════════════════════════════╪════════════════════════════════╪════════════════════════════════╡
│ R5d8539f1d9a5e857189956bad8eb4 ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["C878f017efe6de2805a953d0ca9b │
│ 8e7b11137a25b12c2a7293fa1b6156 ┆ c4707f4ac2919"]                ┆ 8491274a29290"]                │
│ 2e629                          ┆                                ┆                                │
│ R6a89aaf90529aa474f537c081d71f ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["C54130f1c76aaa5380fa631a6a65 │
│ f584fa0179bdde8269c9ae4b80b2e5 ┆ c4707f4ac2919"]                ┆ 9

In [3]:
df.write_parquet("../data/MINES/reactions_compounds_list.parquet")
df

shape: (3_435_078, 3)
┌────────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│ reaction_id                    ┆ starting_compounds             ┆ predicted_compounds            │
│ ---                            ┆ ---                            ┆ ---                            │
│ str                            ┆ list[str]                      ┆ list[str]                      │
╞════════════════════════════════╪════════════════════════════════╪════════════════════════════════╡
│ R5d8539f1d9a5e857189956bad8eb4 ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["C878f017efe6de2805a953d0ca9b │
│ 8e7b11137a25b12c2a7293fa1b6156 ┆ c4707f4ac2919"]                ┆ 8491274a29290"]                │
│ 2e629                          ┆                                ┆                                │
│ R6a89aaf90529aa474f537c081d71f ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["C54130f1c76aaa5380fa631a6a65 │
│ f584fa0179bdde8269c9ae4b80b2e5 ┆ c4707f4ac2919"]                ┆ 9121284978c5d"]                │
│ c96f7                          ┆                                ┆                                │
│ Rf0c39549766c89963dbb1a98f8f1d ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["Cfa5e885b86c8c37a465cad5238e │
│ 4b89431cf4ee60f962c7703ef9df5b ┆ c4707f4ac2919"]                ┆ d62672498a45d"]                │
│ cb327                          ┆                                ┆                                │
│ Rad931d485dae8cbc9aa07c1301163 ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["C57a73b796ef9de341670ad4f895 │
│ f00258690adddbc7279de2b5c92ebc ┆ c4707f4ac2919"]                ┆ 779c4ce0d4623"]                │
│ a656f                          ┆                                ┆                                │
│ R02b3ad62ed7e42f819c6931a7d392 ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["C8e1b680b68eec30be34c6b4857d │
│ b90a5478a1791f2d02145fb0d73fc3 ┆ c4707f4ac2919"]                ┆ 630e2c245759d"]                │
│ 1b151                          ┆                                ┆                                │
│ R74fde49e6e163ead723cc76900847 ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["Cc65bdd68f5ca4038b80b1eb2be0 │
│ 801f1ddf2a6272bd405ab1cba8fa06 ┆ c4707f4ac2919"]                ┆ d5434f75156dc"]                │
│ 0de97                          ┆                                ┆                                │
│ R03894cee5bc4149f878b0ef38cbf2 ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["Cb969d7dca7f60bdb827dc3efb4f │
│ 0535406285eaed4fffdf8825fd2da2 ┆ c4707f4ac2919"]                ┆ c22789592ef6e"]                │
│ 05801                          ┆                                ┆                                │
│ Rd7278d127735d1801a3017dba5ae8 ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["C037f60c0477191d54119504e897 │
│ 70010a33b029c9c1991907db9edebb ┆ c4707f4ac2919"]                ┆ 6d0d64d8c8ce6"]                │
│ 67f6c                          ┆                                ┆                                │
│ R57fbe72efaa8f7f21c5a7449fb071 ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["Cceca72da2195f029e035132a30c │
│ 9490e751f955ad08d793b1628b65bd ┆ c4707f4ac2919"]                ┆ 3ae5b2f5b68b0"]                │
│ 5f263                          ┆                                ┆                                │
│ Rfdba4f6c149488605e09370cb664e ┆ ["Ce8bc5cd3aa30776ab6d35fdc2bc ┆ ["C00eb38722ea74e3671d815c444f │
│ fbd7f4aa514de07bc4808bf5b07b18 ┆ c4707f4ac2919"]                ┆ 48e0db6315491"]                │
│ ad02b                          ┆                                ┆                                │
│ R052499a5977f48c2230ee020ffd21 ┆ ["Cf3ff883e2601ad5ff1487b92aae ┆ ["C06298570e94bc87e59a16459ad1 │
│ 29d38e15ca15b064913e257c0c3ebf ┆ 0bc640572b398"]                ┆ 1a157259d5521"]                │
│ 5313d                          ┆                                ┆                                │
│ Rb70035d1eda7ef8e0e40fca2d795c ┆ ["Cf3ff883e2601ad5ff1487b92aae ┆ ["Cac3b1724a01566dcd8d1bccda78 │
│ 10c72756436c5f7310d9f8cf26ba65 ┆ 0bc6

## save all the entries with type in a list

In [4]:
%%time

from pymongo import MongoClient
import polars as pl

# Set the maximum number of characters displayed per column
pl.Config.set_tbl_rows(20)
pl.Config(fmt_str_lengths=700)

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/') 
db = client['lotus_mines_enzymatic'] 
reactions_collection = db['reactions'] 
    
pipeline = [
    {
        '$project': {
            '_id': 1, 
            'reactants': {
                '$map': {
                    'input': '$Reactants', 
                    'as': 'reactant', 
                    'in': {
                        '$arrayElemAt': [
                            '$$reactant', 1
                        ]
                    }
                }
            }, 
            'products': {
                '$map': {
                    'input': '$Products', 
                    'as': 'product', 
                    'in': {
                        '$arrayElemAt': [
                            '$$product', 1
                        ]
                    }
                }
            }
        }
    }, {
        '$lookup': {
            'from': 'compounds', 
            'localField': 'reactants', 
            'foreignField': '_id', 
            'as': 'reactant_details'
        }
    }, {
        '$lookup': {
            'from': 'compounds', 
            'localField': 'products', 
            'foreignField': '_id', 
            'as': 'product_details'
        }
    }, {
        '$addFields': {
            'reactants': {
                '$map': {
                    'input': '$reactant_details', 
                    'as': 'detail', 
                    'in': {
                        'id': '$$detail._id', 
                        'type': '$$detail.Type'
                    }
                }
            }, 
            'products': {
                '$map': {
                    'input': '$product_details', 
                    'as': 'detail', 
                    'in': {
                        'id': '$$detail._id', 
                        'type': '$$detail.Type'
                    }
                }
            }
        }
    }, {
        '$project': {
            'reactant_details': 0, 
            'product_details': 0
        }
    }
]



shape: (3_435_078, 3)
┌────────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│ reaction_id                    ┆ reactants                      ┆ products                       │
│ ---                            ┆ ---                            ┆ ---                            │
│ str                            ┆ list[struct[2]]                ┆ list[struct[2]]                │
╞════════════════════════════════╪════════════════════════════════╪════════════════════════════════╡
│ R5d8539f1d9a5e857189956bad8eb4 ┆ [{"Ce8bc5cd3aa30776ab6d35fdc2b ┆ [{"C878f017efe6de2805a953d0ca9 │
│ 8e7b11137a25b12c2a7293fa1b6156 ┆ cc4707f4ac2919","Starting      ┆ b8491274a29290","Predicted"},  │
│ 2e629                          ┆ Compound"}, {"X0eb45233dd43eca ┆ {"X73bc8ef21db580aefe4dbc0af17 │
│                                ┆ cb9fb1e31140450e1dace01c5","Co ┆ d4013961d9d17","Coreactant"}]  │
│                                ┆ reactant"}, {"X8dc023d8052d83f ┆  

In [ ]:
# Execute the aggregation pipeline
cursor = reactions_collection.aggregate(pipeline)
data = list(cursor)

# Convert to a Polars DataFrame
df = pl.DataFrame(data)
df = df.rename({"_id": "reaction_id"})

# Show the DataFrame
print(df)

In [5]:
df.write_parquet("../data/MINES/reactions_compounds_list_full.parquet")
df

shape: (3_435_078, 3)
┌────────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│ reaction_id                    ┆ reactants                      ┆ products                       │
│ ---                            ┆ ---                            ┆ ---                            │
│ str                            ┆ list[struct[2]]                ┆ list[struct[2]]                │
╞════════════════════════════════╪════════════════════════════════╪════════════════════════════════╡
│ R5d8539f1d9a5e857189956bad8eb4 ┆ [{"Ce8bc5cd3aa30776ab6d35fdc2b ┆ [{"C878f017efe6de2805a953d0ca9 │
│ 8e7b11137a25b12c2a7293fa1b6156 ┆ cc4707f4ac2919","Starting      ┆ b8491274a29290","Predicted"},  │
│ 2e629                          ┆ Compound"}, {"X0eb45233dd43eca ┆ {"X73bc8ef21db580aefe4dbc0af17 │
│                                ┆ cb9fb1e31140450e1dace01c5","Co ┆ d4013961d9d17","Coreactant"}]  │
│                                ┆ reactant"}, {"X8dc023d8052d83f ┆                                │
│                                ┆ b6feadf8541387e57c199cad0","Co ┆                                │
│                                ┆ reactant"}]                    ┆                                │
│ R6a89aaf90529aa474f537c081d71f ┆ [{"Ce8bc5cd3aa30776ab6d35fdc2b ┆ [{"C54130f1c76aaa5380fa631a6a6 │
│ f584fa0179bdde8269c9ae4b80b2e5 ┆ cc4707f4ac2919","Starting      ┆ 59121284978c5d","Predicted"},  │
│ c96f7                          ┆ Compound"}, {"X0eb45233dd43eca ┆ {"X73bc8ef21db580aefe4dbc0af17 │
│                                ┆ cb9fb1e31140450e1dace01c5","Co ┆ d4013961d9d17","Coreactant"}]  │
│                                ┆ reactant"}, {"X8dc023d8052d83f ┆                                │
│                                ┆ b6feadf8541387e57c199cad0","Co ┆                                │
│                                ┆ reactant"}]                    ┆                                │
│ Rf0c39549766c89963dbb1a98f8f1d ┆ [{"Ce8bc5cd3aa30776ab6d35fdc2b ┆ [{"Cfa5e885b86c8c37a465cad5238 │
│ 4b89431cf4ee60f962c7703ef9df5b ┆ cc4707f4ac2919","Starting      ┆ ed62672498a45d","Predicted"},  │
│ cb327                          ┆ Compound"}, {"X0eb45233dd43eca ┆ {"X73bc8ef21db580aefe4dbc0af17 │
│                                ┆ cb9fb1e31140450e1dace01c5","Co ┆ d4013961d9d17","Coreactant"}]  │
│                                ┆ reactant"}, {"X8dc023d8052d83f ┆                                │
│                                ┆ b6feadf8541387e57c199cad0","Co ┆                                │
│                                ┆ reactant"}]                    ┆                                │
│ Rad931d485dae8cbc9aa07c1301163 ┆ [{"Ce8bc5cd3aa30776ab6d35fdc2b ┆ [{"C57a73b796ef9de341670ad4f89 │
│ f00258690adddbc7279de2b5c92ebc ┆ cc4707f4ac2919","Starting      ┆ 5779c4ce0d4623","Predicted"},  │
│ a656f                          ┆ Compound"}, {"X0eb45233dd43eca ┆ {"X73bc8ef21db580aefe4dbc0af17 │
│                                ┆ cb9fb1e31140450e1dace01c5","Co ┆ d4013961d9d17","Coreactant"}]  │
│                                ┆ reactant"}, {"X8dc023d8052d83f ┆                                │
│                                ┆ b6feadf8541387e57c199cad0","Co ┆                                │
│                                ┆ reactant"}]                    ┆                                │
│ R02b3ad62ed7e42f819c6931a7d392 ┆ [{"Ce8bc5cd3aa30776ab6d35fdc2b ┆ [{"C8e1b680b68eec30be34c6b4857 │
│ b90a5478a1791f2d02145fb0d73fc3 ┆ cc4707f4ac2919","Starting      ┆ d630e2c245759d","Predicted"},  │
│ 1b151                          ┆ Compound"}, {"X0eb45233dd43eca ┆ {"X73bc8ef21db580aefe4dbc0af17 │
│                                ┆ cb9fb1e31140450e1dace01c5","Co ┆ d4013961d9d17","Coreactant"}]  │
│                                ┆ reactant"}, {"X8dc023d8052d83f ┆                                │
│                                ┆ b6feadf8541387e57c199cad0","Co ┆                                │
│                                ┆ reac

## save all the entries with type and counts in a list (df)

In [1]:
%%time

from pymongo import MongoClient
import polars as pl

# Set the maximum number of characters displayed per column
pl.Config.set_tbl_rows(20)
pl.Config(fmt_str_lengths=700)

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/') 
db = client['lotus_mines_enzymatic'] 
reactions_collection = db['reactions'] 
    
pipeline = [
    {
        '$project': {
            '_id': 1, 
            'reactants': {
                '$map': {
                    'input': '$Reactants', 
                    'as': 'reactant', 
                    'in': {
                        '$arrayElemAt': [
                            '$$reactant', 1
                        ]
                    }
                }
            }, 
            'reactants_count': {
                '$map': {
                    'input': '$Reactants', 
                    'as': 'reactant', 
                    'in': {
                        '$arrayElemAt': [
                            '$$reactant', 0
                        ]
                    }
                }
            }, 
            'products': {
                '$map': {
                    'input': '$Products', 
                    'as': 'product', 
                    'in': {
                        '$arrayElemAt': [
                            '$$product', 1
                        ]
                    }
                }
            }, 
            'products_count': {
                '$map': {
                    'input': '$Products', 
                    'as': 'product', 
                    'in': {
                        '$arrayElemAt': [
                            '$$product', 0
                        ]
                    }
                }
            }
        }
    }, {
        '$lookup': {
            'from': 'compounds', 
            'localField': 'reactants', 
            'foreignField': '_id', 
            'as': 'reactant_details'
        }
    }, {
        '$lookup': {
            'from': 'compounds', 
            'localField': 'products', 
            'foreignField': '_id', 
            'as': 'product_details'
        }
    }, {
        '$addFields': {
            'reactants': {
                '$map': {
                    'input': '$reactant_details', 
                    'as': 'detail', 
                    'in': {
                        'id': '$$detail._id', 
                        'type': '$$detail.Type'
                    }
                }
            }, 
            'products': {
                '$map': {
                    'input': '$product_details', 
                    'as': 'detail', 
                    'in': {
                        'id': '$$detail._id', 
                        'type': '$$detail.Type'
                    }
                }
            }
        }
    }, {
        '$project': {
            'reactant_details': 0, 
            'product_details': 0
        }
    }, {
        '$project': {
            'reactants': {
                '$map': {
                    'input': {
                        '$zip': {
                            'inputs': [
                                '$reactants.id', '$reactants.type', '$reactants_count'
                            ]
                        }
                    }, 
                    'as': 'pair', 
                    'in': {
                        'id': {
                            '$arrayElemAt': [
                                '$$pair', 0
                            ]
                        }, 
                        'type': {
                            '$arrayElemAt': [
                                '$$pair', 1
                            ]
                        }, 
                        'count': {
                            '$arrayElemAt': [
                                '$$pair', 2
                            ]
                        }
                    }
                }
            }, 
            'products': {
                '$map': {
                    'input': {
                        '$zip': {
                            'inputs': [
                                '$products.id', '$products.type', '$products_count'
                            ]
                        }
                    }, 
                    'as': 'pair', 
                    'in': {
                        'id': {
                            '$arrayElemAt': [
                                '$$pair', 0
                            ]
                        }, 
                        'type': {
                            '$arrayElemAt': [
                                '$$pair', 1
                            ]
                        }, 
                        'count': {
                            '$arrayElemAt': [
                                '$$pair', 2
                            ]
                        }
                    }
                }
            }
        }
    }
]

CPU times: user 56.5 ms, sys: 24.7 ms, total: 81.2 ms
Wall time: 80.4 ms


In [2]:
%%time

# Execute the aggregation pipeline
cursor = reactions_collection.aggregate(pipeline)
data = list(cursor)

# Convert to a Polars DataFrame
df = pl.DataFrame(data)
df = df.rename({"_id": "reaction_id"})

# Show the DataFrame
print(df)

shape: (3_435_078, 3)
┌────────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│ reaction_id                    ┆ reactants                      ┆ products                       │
│ ---                            ┆ ---                            ┆ ---                            │
│ str                            ┆ list[struct[3]]                ┆ list[struct[3]]                │
╞════════════════════════════════╪════════════════════════════════╪════════════════════════════════╡
│ R5d8539f1d9a5e857189956bad8eb4 ┆ [{"Ce8bc5cd3aa30776ab6d35fdc2b ┆ [{"C878f017efe6de2805a953d0ca9 │
│ 8e7b11137a25b12c2a7293fa1b6156 ┆ cc4707f4ac2919","Starting      ┆ b8491274a29290","Predicted",1} │
│ 2e629                          ┆ Compound",2}, {"X0eb45233dd43e ┆ , {"X73bc8ef21db580aefe4dbc0af │
│                                ┆ cacb9fb1e31140450e1dace01c5"," ┆ 17d4013961d9d17","Coreactant", │
│                                ┆ Coreactant",1}, {"X8dc023d8052 ┆ 1

In [3]:
df.write_parquet("../data/MINES/reactions_compounds_list_with_counts.parquet")
